In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyp
import seaborn as sns
%matplotlib inline

C:\Users\IMOE001\AppData\Local\Temp\ipykernel_14520\163558748.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
legitimate_url_df = pd.read_csv("../Input/data_legitimate_36400.csv")
col_names = ["index", "url"]
legitimate_url_df.columns = col_names

legitimate_url_df = list(legitimate_url_df["url"])
first_50_legitimate_url_df = legitimate_url_df[:50]
print(first_50_legitimate_url_df)

['http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-google', 'https://www.edrawsoft.com/Local-Area-Network.php', 'http://www.webopedia.com/TERM/L/local_area_network_LAN.html', 'https://www.acsac.org/secshelf/book001/16.pdf', 'http://www.diffen.com/difference/LAN_vs_WAN', 'http://sentence.yourdictionary.com/lan', 'http://www.itrelease.com/2012/12/examples-and-types-of-networks/', 'http://www.dictionary.com/browse/lan', 'http://fcit.usf.edu/Network/chap1/chap1.htm', 'http://www.Cisco.com/c/en/us/support/wireless/2100-series-wireless-lan-controllers/products-configuration-examples-list.html', 'http://support.dlink.com/emulators/dcw725/RgConnect.html', 'http://www.maxis.com.my/content/dam/maxis/en/personal/support/faq-images/devices/all-internet-devices/fibre/user-manual/fiber%20internet-%20MB%20TG784%20User%20Manual.pdf', 'http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.178.31&rep=rep1&type=pdf', 'http://www.Cisco.com/c/dam/en/us/td/docs/video/at_home/Cable_Modems/3800_S

In [26]:
import domain_parser

parsed_urls = domain_parser.parse(first_50_legitimate_url_df, "legitimate")
print(parsed_urls)

[{'url': 'http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-google', 'domain': 'conceptdraw', 'registered_domain': 'conceptdraw.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 0, 'protocol': 'http', 'path': '/How-To-Guide/Local-Area-Network-google', 'words_raw': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'google', 'www']}, {'url': 'https://www.edrawsoft.com/Local-Area-Network.php', 'domain': 'edrawsoft', 'registered_domain': 'edrawsoft.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 1, 'protocol': 'https', 'path': '/Local-Area-Network.php', 'words_raw': ['edrawsoft', 'local', 'area', 'network', 'php', 'www']}, {'url': 'http://www.webopedia.com/TERM/L/local_area_network_LAN.html', 'domain': 'webopedia', 'registered_domain': 'webopedia.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 2, 'protocol': 'http', 'path': '/TERM/L/local_area_network_LAN.html', 'words_raw': ['webopedia', 'term', 'l', 'lo

In [27]:
from dataPreProcessing import filter_brands
from dataPreProcessing import subtract_lists

for parsed_url in parsed_urls:
    # brands
    parsed_url["words_raw_for_filters"] = list(parsed_url["words_raw"]) # remove from all dicts at the end
    parsed_url["brand_words"] = filter_brands(parsed_url["words_raw_for_filters"])
    parsed_url["words_raw_for_filters"] = subtract_lists(parsed_url["words_raw_for_filters"], parsed_url["brand_words"])



print(parsed_urls)

[{'url': 'http://www.conceptdraw.com/How-To-Guide/Local-Area-Network-google', 'domain': 'conceptdraw', 'registered_domain': 'conceptdraw.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 0, 'protocol': 'http', 'path': '/How-To-Guide/Local-Area-Network-google', 'words_raw': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'google', 'www'], 'words_raw_for_filters': ['conceptdraw', 'how', 'to', 'guide', 'local', 'area', 'network', 'www'], 'brand_words': ['google']}, {'url': 'https://www.edrawsoft.com/Local-Area-Network.php', 'domain': 'edrawsoft', 'registered_domain': 'edrawsoft.com', 'tld': 'com', 'subdomain': 'www', 'class': 'legitimate', 'id': 1, 'protocol': 'https', 'path': '/Local-Area-Network.php', 'words_raw': ['edrawsoft', 'local', 'area', 'network', 'php', 'www'], 'words_raw_for_filters': ['edrawsoft', 'local', 'area', 'network', 'php', 'www'], 'brand_words': []}, {'url': 'http://www.webopedia.com/TERM/L/local_area_network_LAN.html', 'domain': 